In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
data_1 = pd.read_excel('综合平均评分.xlsx',sheet_name='红葡萄酒')
data_2 = pd.read_excel('综合平均评分.xlsx',sheet_name='白葡萄酒')
data_1.head()

,酒样品,第一组评分,第二组评分
0,1,62.7,68.1
1,2,80.3,74.0
2,3,80.4,74.6
3,4,68.6,71.2
4,5,73.3,72.1


## 正态分布检验：Shapior-wilk检验

In [7]:
from scipy import stats
"""
Shapior-wilk检验主要应用于小样本(3<n<50)的情况;
H0: 该变量服从正态分布,H1: 该变量不服从正态分布;
"""
shapiro_test = stats.shapiro(data_1['第一组评分'])
print("Shapiro wilk Test Statstic:{}   Pvalue:{}".format(shapiro_test.statistic,shapiro_test.pvalue))
shapiro_test = stats.shapiro(data_1['第二组评分'])
print("Shapiro wilk Test Statstic:{}   Pvalue:{}".format(shapiro_test.statistic,shapiro_test.pvalue))
shapiro_test = stats.shapiro(data_2['第一组评分'])
print("Shapiro wilk Test Statstic:{}   Pvalue:{}".format(shapiro_test.statistic,shapiro_test.pvalue))
shapiro_test = stats.shapiro(data_2['第二组评分'])
print("Shapiro wilk Test Statstic:{}   Pvalue:{}".format(shapiro_test.statistic,shapiro_test.pvalue))

Shapiro wilk Test Statstic:0.9248942732810974   Pvalue:0.05186120793223381
Shapiro wilk Test Statstic:0.9809100031852722   Pvalue:0.882304847240448
Shapiro wilk Test Statstic:0.9789304137229919   Pvalue:0.824052095413208
Shapiro wilk Test Statstic:0.9455091953277588   Pvalue:0.15258711576461792
